In [1]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import shap

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

np.random.seed(1)

In [2]:
## Train the model

# Import dataset
filepath = os.path.join('..', 'datasets', 'brand_meaning.csv')
df = pd.read_csv(filepath)

# Drop `ID` column
df = df.drop(columns=['employee_id'])
df.head()

,company_mission,company_vision,company_values,mission_vision_values_on_my_work,company face_clients_competitors_media,company_distinguish_competitors,customer_needs_fulfillment,company_visual_presentation,knowing_customer_promise_fulfillment,knowing_meet_user_expectations,clear_customers_feel,knowing_customers,sales_dep,marketing_dep,services_dep,finance_dep,target
0,4,3,3,5,5,3,4,5,5,5,4,5,1,0,0,0,1
1,4,4,4,4,4,5,5,5,5,5,5,5,1,0,0,0,1
2,4,4,4,4,5,4,4,2,4,5,5,4,1,0,0,0,0
3,2,3,5,5,3,5,2,4,5,4,5,5,1,0,0,0,0
4,4,3,4,4,5,5,3,4,5,5,4,4,1,0,0,0,0


In [3]:
# Get class balance of 'target' col
df['target'].value_counts()

target
0    540
1    460
Name: count, dtype: int64

In [4]:
# 1. Isolate X variables
X = df.drop(columns=['target'])

# 2. Isolate y variable
y = df['target']

# 3. Split into train and test sets
X_tr, X_test, y_tr, y_test = train_test_split(X, y, stratify=y,
                                              test_size=0.15, random_state=42)

# 4. Split into train and validate sets
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, stratify=y_tr,
                                                  test_size=0.17, random_state=42)

# Print class distributions
print("Train class distribution:", np.unique(y_train, return_counts=True))
print("Validation class distribution:", np.unique(y_val, return_counts=True))
print("Test class distribution:", np.unique(y_test, return_counts=True))

Train class distribution: (array([0, 1], dtype=int64), array([381, 324], dtype=int64))
Validation class distribution: (array([0, 1], dtype=int64), array([78, 67], dtype=int64))
Test class distribution: (array([0, 1], dtype=int64), array([81, 69], dtype=int64))


In [5]:
for x in [X_train, X_val, X_test]:
    print(len(x))

705
145
150


In [6]:
# Min-Max Scaling
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_train = X_train_scaled

X_val_scaled = scaler.transform(X_val)
X_val = X_val_scaled

X_test_scaled = scaler.transform(X_test)
X_test = X_test_scaled

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (705, 16)
Shape of y_train: (705,)
Shape of X_val: (145, 16)
Shape of y_val: (145,)
Shape of X_test: (150, 16)
Shape of y_test: (150,)


In [7]:
n_x = 16      # number of input features
n_y = 1       # number of ouputs

In [8]:
def predict(n_h1, n_h2):
    
    # Define the input layer separately
    input_layer = layers.Input(shape=(n_x,))  # Define the input layer explicitly

    # Define the rest of the model
    x = layers.Dense(n_h1, activation='relu')(input_layer)  # First hidden layer
    x = layers.Dense(n_h2, activation='relu')(x)  # Second hidden layer
    output_layer = layers.Dense(n_y, activation='sigmoid')(x)  # Output layer (binary classification)

    # Create the model by specifying inputs and outputs
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    # Train the model
    history = model.fit(
        X_train, y_train, 
        epochs=150, 
        batch_size=16, 
        validation_data=(X_val, y_val),
        verbose=0
    )

    # Training set

    # Get predictions
    y_train_pred_probs = model.predict(X_train)  # Probabilities
    y_train_pred = (y_train_pred_probs >= 0.5).astype(int)   # Convert to binary predictions

    # Compute metrics
    precision_train = precision_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    f1_train = f1_score(y_train, y_train_pred)
    accuracy_train = accuracy_score(y_train, y_train_pred)
    roc_auc_train = roc_auc_score(y_train, y_train_pred_probs)

    print(f"Training Metrics:")
    print(f"Precision: {precision_train:.4f}")
    print(f"Recall: {recall_train:.4f}")
    print(f"F1: {f1_train:.4f}")
    print(f"Accuracy: {accuracy_train:.4f}")
    print(f"ROC AUC: {roc_auc_train:.4f}")


    # Validation set

    # Get predictions
    y_val_pred_probs = model.predict(X_val)  # Probabilities
    y_val_pred = (y_val_pred_probs >= 0.5).astype(int)  # Convert to binary predictions

    # Compute metrics
    precision_val = precision_score(y_val, y_val_pred)
    recall_val = recall_score(y_val, y_val_pred)
    f1_val = f1_score(y_val, y_val_pred)
    accuracy_val = accuracy_score(y_val, y_val_pred)
    roc_auc_val = roc_auc_score(y_val, y_val_pred_probs)

    print(f"Validation Metrics:")
    print(f"Precision: {precision_val:.4f}")
    print(f"Recall: {recall_val:.4f}")
    print(f"F1: {f1_val:.4f}")
    print(f"Accuracy: {accuracy_val:.4f}")
    print(f"ROC AUC: {roc_auc_val:.4f}")



    # Test set

    # Get predictions
    y_test_pred_probs = model.predict(X_test)  # Probabilities
    y_test_pred = (y_test_pred_probs >= 0.5).astype(int)  # Convert to binary predictions

    # Compute metrics
    precision_test = precision_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    f1_test = f1_score(y_test, y_test_pred)
    accuracy_test = accuracy_score(y_test, y_test_pred)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_probs)

    print(f"Test Metrics:")
    print(f"Precision: {precision_test:.4f}")
    print(f"Recall: {recall_test:.4f}")
    print(f"F1: {f1_test:.4f}")
    print(f"Accuracy: {accuracy_test:.4f}")
    print(f"ROC AUC: {roc_auc_test:.4f}")

    return model, accuracy_test, precision_test, recall_test, f1_test, roc_auc_test


In [17]:
n_h1=0
n_h2=0
best_results = {
    'accuracy' : 0,
    'precision' : 0,
    'recall' : 0,
    'f1' : 0,
    'roc_auc' : 0
}
sum_best_results = 0
h1_candidates = [2 * n_x, 3 * n_x, 4 * n_x]
h2_candidates = [h1 // 2 for h1 in h1_candidates]

for h1, h2 in zip(h1_candidates, h2_candidates):
    print(f"Training FNN with H1={h1}, H2={h2}")
    
    model, accuracy, precision, recall, f1, roc_auc = predict(h1, h2)
    
    sum_results = accuracy + precision + recall + f1 + roc_auc
    
    if sum_results > sum_best_results:
        sum_best_results = sum_results
        best_results['roc_auc'] = roc_auc
        best_results['accuracy'] = accuracy
        best_results['precision'] = precision
        best_results['recall'] = recall
        best_results['f1'] = f1
        n_h1 = h1
        n_h2 = h2
            

print(f'Best test results come from FNN ({n_x},{n_h1},{n_h2},{n_y})')

print(f'Precision: {best_results["precision"]:.4f}')
print(f'Recall: {best_results["recall"]:.4f}')
print(f'F1 Score: {best_results["f1"]:.4f}')
print(f'Accuracy: {best_results["accuracy"]:.4f}')
print(f'ROC_AUC: {best_results["roc_auc"]:.4f}')

results_df = pd.DataFrame([best_results])  # Use [best_results] to make it a single-row DataFrame

# Step 3: Export to Excel
results_df.to_excel('bc_best_FNN_results.xlsx', index=False)

# Optional: Display DataFrame
print(results_df)


Training FNN with H1=32, H2=16
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Training Metrics:
Precision: 1.0000
Recall: 1.0000
F1: 1.0000
Accuracy: 1.0000
ROC AUC: 1.0000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Validation Metrics:
Precision: 0.8571
Recall: 0.8955
F1: 0.8759
Accuracy: 0.8828
ROC AUC: 0.9705
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Test Metrics:
Precision: 0.9531
Recall: 0.8841
F1: 0.9173
Accuracy: 0.9267
ROC AUC: 0.9809
Training FNN with H1=48, H2=24
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Training Metrics:
Precision: 1.0000
Recall: 1.0000
F1: 1.0000
Accuracy: 1.0000
ROC AUC: 1.0000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Validation Metrics:
Precision: 0.8594
Recall: 0.8209
F1: 0.8397
Accuracy: 0.8552
ROC AUC: 0.9684
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Test Metrics:
Precision: 0.9828
Recall: 0.8261
F1: 0.8976
Accuracy: 0.9133
ROC AUC: 0.9855
Training FNN with H1=64, H2=32
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training Metrics:
Precision: 1.0000
Recall: 0.9969
F1: 0.9985
Accuracy:

In [ ]:
def shap_to_probability_delta(shap_val):
    return np.tanh(shap_val)  # maps raw SHAP value to ~[-1, 1]

In [ ]:
# Sample background data from training set (DeepExplainer needs this for reference)
background = X_train[:100]

# Initialize SHAP DeepExplainer
explainer = shap.DeepExplainer(model, background)

# Compute SHAP values for the dataset you want to explain (usually X_train or X_test)
shap_values = explainer.shap_values(X_train)  # Ensure data is in NumPy format

# Extract SHAP values for class 1 (positive class) from shap_values
shap_values_class_1 = shap_values[:, :, 0]

# Compute the mean SHAP value per feature (across all samples)
mean_shap_values = np.mean(shap_values_class_1, axis=0)  # Mean SHAP value per feature for positive class

# Compute mean absolute SHAP values per feature
mean_abs_shap_values = np.mean(np.abs(shap_values_class_1), axis=0)  # Mean absolute SHAP values per feature

# Direction: 'Increases' if the mean SHAP value for a feature is positive, 'Decreases' otherwise
direction = ['Inc.' if val > 0 else 'Dec.' for val in mean_shap_values]

# Binning the impact into categories (Low, Medium, High) based on quantiles of the mean absolute SHAP values
impact_strength = pd.qcut(mean_abs_shap_values, q=3, labels=["Low", "Medium", "High"])

# Create a DataFrame to summarize the SHAP values for each feature
shap_summary_df = pd.DataFrame({
'Feature': X.columns,  # Features in the dataset
'Imp. %': np.round(shap_to_probability_delta(mean_abs_shap_values)*100, 1),  # Impact percentage for each feature
'Imp. Direct.': direction,  # Impact direction based on mean SHAP values
'Imp. Strength': impact_strength  # Categorized impact strength (Low, Medium, High)
}).sort_values(by=['Imp. Direct.', 'Imp. %'], ascending=False).reset_index(drop=True)

# Save the SHAP summary to Excel for further inspection
shap_summary_df.to_excel('bm_shap_summary_fnn.xlsx', index=False)

# Display the summary DataFrame
shap_summary_df
